In [ ]:
import pymysql
import requests
from bs4 import BeautifulSoup

### Crawl College Numbers

In [ ]:
url = 'http://ysweb.yonsei.ac.kr:8888/curri120601/curri_new.jsp#top'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
college_tags = soup.select('#OCODE1 option')
college_names = []
college_nos = []
for college_tag in college_tags:
    college_names.append(college_tag.text)
    college_nos.append(college_tag.attrs['value'])

### searchKey_crawler

In [ ]:
def searchkey_crawler(college_no,year,semester):
    url = "http://ysweb.yonsei.ac.kr:8888/DataAgent?pgm=/curri120601/curri_Data&DmlGb=get_list&ocode0=s1&ocode1="+college_no+"&s2=all&schHakjum=all&hy="+year+"&hg="+semester+"&lang=0&filterscount=0&groupscount=0&pagenum=0&pagesize=15&recordstartindex=0&recordendindex=34.56"
    header =  {"Referer" : "http://ysweb.yonsei.ac.kr:8888/curri120601/curri_new.jsp"}
    response = requests.get(url, headers = header)
    result = []
    for data in response.json()['Data']:
        searchdict = {'yshs_domain': data['DOMAIN'],
                      'yshs_hyhg': data['HYHG'],
                      'yshs_hakno': data['HAKBBSBB'].split('-')[0],
                      'yshs_bb': data['HAKBBSBB'].split('-')[1],
                      'yshs_sbb': data['HAKBBSBB'].split('-')[2]}
        result.append(searchdict)
    return(result)

### page_crawler

In [ ]:
def page_crawler(params):
    url= "http://ysweb.yonsei.ac.kr:8888/curri120601/curri_pop_mileage_result01.jsp"
    response = requests.post(url=url,data=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    syllabus = []
    if len(soup.findAll('tr')) < 4:
        return [], []
    for tds in soup.findAll('tr')[4].findAll('td'):
        syllabus.append(tds.text)
    mileages = []
    for trs in soup.findAll('tr')[6].findAll('tr'):
        row = [params['yshs_domain'],params['yshs_hyhg'],params['yshs_hakno'],params['yshs_bb'],params['yshs_sbb']]
        for tds in trs.findAll('td'):
            row.append(tds.text)
        mileages.append(row)
    return syllabus, mileages

### Create Database `sugang_db`

In [ ]:
myhost = 'localhost'
myport = 3306
myuser = 'root'
mypasswd = 'password'
mydb = 'sugang_db'

In [ ]:
conn = pymysql.connect(
    host=myhost,
    port=myport,
    user=myuser,
    password=mypasswd,
    charset='utf8'
)

try:
    with conn.cursor() as cursor:
        sql = "CREATE DATABASE " + mydb
        cursor.execute(sql)
        sql = "SHOW DATABASES"
        cursorInstance.execute(sql)
        dbList = cursorInstance.fetchall()
        print(dbList)
except Exception as err:
    print("Error:", err)
finally:
    conn.close()

### Create Table `syllabi`

In [ ]:
conn = pymysql.connect(
    host=myhost, 
    port=myport, 
    user=myuser,
    password=mypasswd,
    db=mydb,
    charset='utf8',
)

try:
    with conn.cursor() as cursor:
        sql = '''CREATE TABLE syllabi (
                hyhg             INT          NOT NULL,
                course_code      VARCHAR(255) NOT NULL,
                course_title     VARCHAR(255) NOT NULL,
                credit           INT          NOT NULL,
                instructor       VARCHAR(255) NOT NULL,
                time             VARCHAR(255) NOT NULL,
                room             VARCHAR(255) NOT NULL,
                quota            INT          NOT NULL,
                participants     VARCHAR(255) NOT NULL,
                major_quota      INT          NOT NULL, 
                second_major     VARCHAR(255) NOT NULL,
                grade_1          INT          NOT NULL,
                grade_2          INT          NOT NULL, 
                grade_3          INT          NOT NULL, 
                grade_4          INT          NOT NULL, 
                exchange_student VARCHAR(255) NOT NULL, 
                max_mileage      INT          NOT NULL, 
                min_result       INT          NOT NULL, 
                max_result       INT          NOT NULL, 
                average          FLOAT        NOT NULL
               )'''
        cursor.execute(sql)
    conn.commit()
except Exception as err:
    print("Error:", err)
finally:
    conn.close()

### Create Table `mileages`

In [ ]:
conn = pymysql.connect(
    host=myhost, 
    port=myport, 
    user=myuser,
    password=mypasswd,
    db=mydb,
    charset='utf8',
)

try:
    with conn.cursor() as cursor:
        sql = '''CREATE TABLE mileages (
                domain                VARCHAR(255) NOT NULL, 
                hyhg                  INT          NOT NULL, 
                hakno                 VARCHAR(255) NOT NULL, 
                bb                    INT          NOT NULL, 
                sbb                   INT          NOT NULL, 
                rank                  INT          NOT NULL, 
                mileages              INT          NOT NULL, 
                major                 VARCHAR(255) NOT NULL,
                double_major          VARCHAR(255) NOT NULL, 
                enrolled_courses      INT          NOT NULL, 
                graduation            VARCHAR(255) NOT NULL, 
                first_enroll          VARCHAR(255) NOT NULL, 
                credits_rate          FLOAT        NOT NULL, 
                previous_credits_rate FLOAT        NOT NULL, 
                grade                 INT          NOT NULL, 
                enrolled              VARCHAR(255) NOT NULL, 
                etc                   VARCHAR(255) NULL
               )'''
        cursor.execute(sql)
    conn.commit()
except Exception as err:
    print("Error:", err)
finally:
    conn.close()

### Show Tables

In [ ]:
conn = pymysql.connect(
    host=myhost,
    port=myport,
    user=myuser,
    password=mypasswd,
    db=mydb,
    charset='utf8'
)

try:
    with conn.cursor() as cursor:
        sql = "SHOW TABLES"
        cursor.execute(sql)
        tableList = cursor.fetchall()
        print(tableList)
except Exception as err:
    print("Error:", err)
finally:
    conn.close()

### Preprocess

In [ ]:
def syllabus_preprocess(syllabus):
    major_split = syllabus[9].split(' ')
    syllabus[9] = major_split[0]
    syllabus.insert(10, major_split[1][1:-1])
    return tuple(syllabus)

In [ ]:
def mileage_preprocess(mileage):
    major_split = mileage[7].split(' ')
    mileage[7] = major_split[0]
    mileage.insert(8, major_split[1][1:-1])
    return tuple(mileage)

### Insert Data into Table `syllabi`

In [ ]:
def insert_syllabus(result):
    try:
        with conn.cursor() as cursor:
            sql = 'INSERT INTO syllabi VALUES (' + '%s, ' * 19 + '%s)'
            cursor.execute(sql, result)
        conn.commit()
    except Exception as err:
        print("Error:", err)

### Insert Data into Table `mileages`

In [ ]:
def insert_mileage(result):
    try:
        with conn.cursor() as cursor:
            sql = 'INSERT INTO mileages VALUES (' + '%s, ' * 16 + '%s)'
            cursor.execute(sql, result)
        conn.commit()
    except Exception as err:
        print("Error:", err)

### Crawling

In [ ]:
years = ['2016', '2017', '2018', '2019', '2020']
semesters = ['1', '2']

In [ ]:
conn = pymysql.connect(
    host=myhost,
    port=myport,
    user=myuser,
    password=mypasswd,
    db=mydb,
    charset='utf8'
)

for year in years:
    for semester in semesters:
        if (year == '2020') and (semester == '2'):
            break
        for college_no in college_nos:
            print(year, semester, college_no)
            searchparams = searchkey_crawler(college_no, year, semester)
            for i in searchparams:
                crawl = page_crawler(i)
                if crawl == -1:
                    continue
                syllabus, mileages = crawl
                if len(syllabus) < 18:
                    continue
                insert_syllabus(syllabus_preprocess([year+semester]+syllabus))
                for mileage in mileages:
                    insert_mileage(mileage_preprocess(mileage))
                    
conn.close()